In [ ]:
!git clone https://github.com/pedropro/TACO.git


Cloning into 'TACO'...
remote: Enumerating objects: 476, done.
remote: Counting objects: 100% (206/206), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 476 (delta 150), reused 124 (delta 71), pack-reused 270
Receiving objects: 100% (476/476), 68.35 MiB | 18.72 MiB/s, done.
Resolving deltas: 100% (301/301), done.


In [ ]:
%cd TACO/
!python download.py

/content/TACO
Note. If for any reason the connection is broken. Just call me again and I will start where I left.
Finished


In [ ]:
%cd ..


/content


In [ ]:
!git clone https://github.com/rkuo2000/yolov5
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 3390, done.
remote: Total 3390 (delta 0), reused 0 (delta 0), pack-reused 3390
Receiving objects: 100% (3390/3390), 7.02 MiB | 32.99 MiB/s, done.
Resolving deltas: 100% (2274/2274), done.
/content/yolov5


In [ ]:
!pip install pycocotools

In [ ]:
!pip install google_utils

In [ ]:
from google_utils import *
attempt_download('weights/yolov5s.pt')
attempt_download('weights/yolov5m.pt')
attempt_download('weights/yolov5l.pt')
attempt_download('weights/yolov5x.pt')

NameError: ignored

In [ ]:
import os
import shutil
import numpy as np
import tqdm
from google.colab import files as locfiles

In [ ]:
from pycocotools.coco import COCO
data_source = COCO(annotation_file='/content/TACO/data/annotations.json')

loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


In [ ]:
# remapping label id to 0~1
# label_transfer = {5: 0, 12: 1}
label_transfer = {5: 0, 12: 1, 7: 2, 29: 3, 36: 4, 39: 5, 58: 6, 59: 7}

In [ ]:
img_ids = data_source.getImgIds()

catIds = data_source.getCatIds()
categories = data_source.loadCats(catIds)
categories.sort(key=lambda x: x['id'])
classes = {}
coco_labels = {}
coco_labels_inverse = {}
for c in categories:
    coco_labels[len(classes)] = c['id']
    coco_labels_inverse[c['id']] = len(classes)
    classes[c['name']] = len(classes)

class_num = {}

In [ ]:
!mkdir -p tmp/labels tmp/images
save_base_path  = 'tmp/labels/'
save_image_path = 'tmp/images/'

In [ ]:
for index, img_id in tqdm.tqdm(enumerate(img_ids), desc='change .json file to .txt file'):
    img_info = data_source.loadImgs(img_id)[0]
    save_name = img_info['file_name'].replace('/', '_')
    file_name = save_name.split('.')[0]
    height = img_info['height']
    width = img_info['width']
    save_path = save_base_path + file_name + '.txt'
    is_exist = False 
    with open(save_path, mode='w') as fp:

        annotation_id = data_source.getAnnIds(img_id)
        boxes = np.zeros((0, 5))
        if len(annotation_id) == 0:  
            fp.write('')
            continue
        
        annotations = data_source.loadAnns(annotation_id)
        lines = ''  
        for annotation in annotations:
           
            label = coco_labels_inverse[annotation['category_id']]
            if label in label_transfer.keys():
                
                is_exist = True
                box = annotation['bbox']
                if box[2] < 1 or box[3] < 1:
                    
                    continue
                # top_x,top_y,width,height==>cen_x,cen_y,width,height
                box[0] = round((box[0] + box[2] / 2) / width, 6)
                box[1] = round((box[1] + box[3] / 2) / height, 6)
                box[2] = round(box[2] / width, 6)
                box[3] = round(box[3] / height, 6)
                label = label_transfer[label]  
                if label not in class_num.keys():
                    class_num[label] = 0
                class_num[label] += 1
                lines = lines + str(label) 
                for i in box:  
                    lines += ' ' + str(i)
                lines += '\n' 
        fp.writelines(lines)
    if is_exist:
        shutil.copy('/content/TACO/data/{}'.format(img_info['file_name']), os.path.join(save_image_path, save_name))
    else:
        #
        os.remove(save_path)

change .json file to .txt file: 1500it [00:07, 206.37it/s]


In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
splitfolders.ratio('tmp', output="taco", seed=1337, ratio=(.8, 0.1,0.1)) 

Copying files: 2172 files [00:07, 293.42 files/s]


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 268 kB 6.9 MB/s 
     |████████████████████████████████| 2.8 MB 44.9 MB/s 
     |████████████████████████████████| 776.7 MB 4.8 kB/s 
     |████████████████████████████████| 12.7 MB 15.6 MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3-cp37-cp37m-linux_x86_64.whl size=44244 sha256=2f35ccde30076cbb1f0de89f40bcf6e5827118aad2880eca26b0928aa9269f58
  Stored in directory: /root/.cache/pip/wheels/8a/55/a4/c0a81d27c33462cfdcb904db018f5550197e88b2b6b85beed2
Successfully built PyYAML
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu111
    Uninstalling torchvision-0.10.0+cu111:
      Successfully uninstalled torchvision-0.10.0+cu111
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-

In [ ]:
%mkdir /content/yolov5-taco-models
!touch /content/yolov5-taco-models/taco8.yaml

In [ ]:
!pip install torch==1.7.0

In [ ]:
pwd

'/content/yolov5'

In [ ]:
!python train.py --img 640 --batch 1 --epochs 10 --data /content/yolov5-taco-models/taco8.yaml --cfg models/yolov5l.yaml --weights yolov5l.pt


Using torch 1.7.0 CUDA:0 (Tesla P100-PCIE-16GB, 16280MB)

Namespace(adam=False, batch_size=1, bucket='', cache_images=False, cfg='models/yolov5l.yaml', data='/content/yolov5-taco-models/taco8.yaml', device='', epochs=10, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', rect=False, resume=False, save_dir='runs/train/exp2', single_cls=False, sync_bn=False, total_batch_size=1, weights='yolov5l.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
Hyperparameters {'lr0': 0.01, 'lrf': 0.2, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'warmup_bias_lr': 0.1, 'box': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 

In [ ]:
!tar -zcvf runs/exp6_complete.tar.gz runs/train/exp/ 
locfiles.download('runs/exp_complete.tar.gz')

In [ ]:
!python train.py --img 640 --batch 1 --epochs 10 --data /content/yolov5-taco-models/taco8.yaml --cfg models/yolov5l.yaml --weights runs/train/exp/weights/yolov5l.pt


In [ ]:
!tar -zcvf runs/exp6_complete.tar.gz runs/train/exp/ 
locfiles.download('runs/exp_complete.tar.gz')

In [ ]:
!python train.py --img 412 --batch 1 --epochs 10 --data /content/yolov5-taco-models/taco8.yaml --cfg models/yolov5s.yaml --weights runs/train/exp2/weights/best.pt


In [ ]:
!python train.py --img 412 --batch 1 --epochs 10 --data /content/yolov5-taco-models/taco8.yaml --cfg models/yolov5s.yaml --weights runs/train/exp3/weights/best.pt


In [ ]:
!tar -zcvf runs/exp4_complete.tar.gz runs/train/exp4/ 
locfiles.download('runs/exp4_complete.tar.gz')

In [ ]:
!python train.py --img 416 --batch 1 --epochs 10 --data /content/yolov5-taco-models/taco8.yaml --cfg models/yolov5s.yaml --weights runs/train/exp4/weights/best.pt


In [ ]:
!tar -zcvf runs/exp5_complete.tar.gz runs/train/exp5/ 
locfiles.download('runs/exp5_complete.tar.gz')

In [ ]:
from IPython.display import Image

In [ ]:
!python detect.py --weights runs/train/exp5/weights/best.pt --img 640 --conf 0.1 --source /content/yolov5/tmp/images/batch_1_000004.jpg

In [ ]:
!python detect.py --weights /content/yolov5/weights/yolov5l.pt --img 640 --conf 0.4 --source /content/yolov5/tmp/images/batch_1_000004.jpg

In [ ]:
Image('runs/detect/exp10/batch_1_000004.jpg')

In [ ]:
!python train.py --img 416 --batch 1 --epochs 10 --data /content/yolov5-taco-models/taco8.yaml --cfg models/yolov5s.yaml --weights runs/train/exp5/weights/best.pt


In [ ]:
!tar -zcvf runs/exp5_complete.tar.gz runs/train/exp6/ 
locfiles.download('runs/exp6_complete.tar.gz')

In [ ]:
!python train.py --img 416 --batch 1 --epochs 10 --data /content/yolov5-taco-models/taco8.yaml --cfg models/yolov5s.yaml --weights runs/train/exp6/weights/best.pt


In [ ]:
!tar -zcvf runs/exp5_complete.tar.gz runs/train/exp7/ 
locfiles.download('runs/exp7_complete.tar.gz')

In [ ]:
!python train.py --img 416 --batch 1 --epochs 10 --data /content/yolov5-taco-models/taco8.yaml --cfg models/yolov5s.yaml --weights runs/train/exp7/weights/best.pt
!tar -zcvf runs/exp5_complete.tar.gz runs/train/exp8/ 
locfiles.download('runs/exp8_complete.tar.gz')

In [ ]:
!python train.py --img 416 --batch 1 --epochs 10 --data /content/yolov5-taco-models/taco8.yaml --cfg models/yolov5s.yaml --weights runs/train/exp8/weights/best.pt
!tar -zcvf runs/exp5_complete.tar.gz runs/train/exp9/ 
locfiles.download('runs/exp9_complete.tar.gz')

In [ ]:
!python train.py --img 416 --batch 1 --epochs 10 --data /content/yolov5-taco-models/taco8.yaml --cfg models/yolov5s.yaml --weights runs/train/exp9/weights/best.pt
!tar -zcvf runs/exp5_complete.tar.gz runs/train/exp10/ 
locfiles.download('runs/exp10_complete.tar.gz')

In [ ]:
% cd TACO/yolov5

/content/TACO/yolov5
